## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

In [1]:
#загрузка данных и библиотек которые потребуются в проектной работе
import numpy as np
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [2]:
#изучение информации
df = pd.read_csv('/home/andrey/Загрузки/data.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Было произведено ознакомление с данными и получено предарительное понимание в каких столбцах датафрейма необходимо внести изменения и корректировки. 
В столбцах 'days_employed' и 'total_income' присутствует достаточно большое количество пропущенных значений.
В некоторых столбцах (напр. пол) есть аномальные значения которые тоже потребуется обработать.
Данные в столбце 'purpose' необходимо будет лемматизировать.

### Обработка пропусков

In [5]:
df = df.drop(df[df["gender"] == 'XNA'].index) #удалим аномальное значение из пола клиента
df = df.drop(df[df["dob_years"] == 0].index) #удалим клиентов с возрастом 0 лет. Т.к. их количество незначительно.
df['children'] = abs(df['children']) #уберём отрицательное значение в количестве детей
df = df.drop(df[df["children"] == 20].index) #удалим аномальные значения с 20 детьми

In [6]:
#приведём к единому регистру и сгрупиируем уровни образования
df['education'] = df['education'].str.lower()
df = df.replace('ученая степень' , 'высшее')
df['education_id'] = df['education_id'].replace(4 , 0)
df = df.replace('начальное' , 'среднее')
df['education_id'] = df['education_id'].replace(3 , 1)


In [7]:
#убираем типы занятости встречающиеся 1-2 раза.
df = df.replace('в декрете' , 'сотрудник')
df = df.replace('предприниматель' , 'компаньон')
df = df.drop(df[df["income_type"] == 'студент'].index)
df = df.drop(df[df["income_type"] == 'безработный'].index)

In [8]:
#рассчитаем медианные значения дохода сгруппированные по типу занятости и образованию. 
#заполним ими пропущенные значения.

df['total_income'] = df.groupby(['income_type'
                                 ,'education'])['total_income'].transform(lambda x: x.fillna(x.median()))

In [9]:
#уберём отрицательное значение в трудовом стаже
df['days_employed'] = abs(df['days_employed'])  

In [10]:
#заполним нулями пропуски в трудовом стаже
df['days_employed'] = df['days_employed'].fillna(0) 

In [11]:
#заменим аномальный трудовой стаж более 60 лет на нули
for i in df['days_employed']:
    if i > 21600:
        i = 0

In [12]:
#заменим нули на NA
df['days_employed'] = df['days_employed'].replace(0, np.nan)

#сгруппируем NA по возрасту и заменим на медианные значения для каждого возраста.
df['days_employed'] = df.groupby(['dob_years'])['days_employed'].transform(lambda x: x.fillna(x.median()))

Пропуски есть только в столбцах 'days_employed' и 'total_income'. При чём если пропущено значение о трудовом стаже, то так же пропущено значение и о текущем доходе. Из этого следует, что данные отсутсвуют из-за того, что клиент ранее по какой либо причине не предоставлял в банк информацию о своих доходах и занятоси. 
В связи с этим, и невозможностью выяснить доход, строки с отсутствующим значением дохода были заменены на медианные значения сгруппированные по типу занятости и уровню образования.
В столбце 'days_employed' аномальные значения и пропущенные значения были заменены на медианные в зависимости от возвраста клиента.

In [13]:
#заменим тим данных float64 на int64
df['days_employed'] = df['days_employed'].astype(int)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21345 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21345 non-null  int64  
 1   days_employed     21345 non-null  int64  
 2   dob_years         21345 non-null  int64  
 3   education         21345 non-null  object 
 4   education_id      21345 non-null  int64  
 5   family_status     21345 non-null  object 
 6   family_status_id  21345 non-null  int64  
 7   gender            21345 non-null  object 
 8   income_type       21345 non-null  object 
 9   debt              21345 non-null  int64  
 10  total_income      21345 non-null  float64
 11  purpose           21345 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB
None


Для лучшей визуализации и из-за отcутствия влияния на результат тип данных float64 меняем на int64, в столбце 'days_employed'

In [14]:
#удаляем дубликаты
df = df.drop_duplicates().reset_index(drop = True)

#проверяем, что дубликаты удалены
if df.duplicated().sum() == 0:
    print('Дубликатов нет')
else:
    print('Не все дубликаты удалены')

Дубликатов нет


Убираем дубликаты данных. Причиной появления дубликатов данных могут быть "задвоенные" заявки от клиентов, например отправленные с сайта банка и из приложения.

In [15]:
#определим основаные лемматезированные цели получения кредита
lemmas_words = []
for i in df['purpose']:
    lemmas = ' '.join(m.lemmatize(i))
    lemmas_words.append(lemmas)
lemmas_counts = Counter(lemmas_words)
print(lemmas_counts)


#изменим данные в столбце purpose исходя из полученной информации
lemmas_words_column = []
for i in lemmas_words:
    if 'свадьба' in i:
        lemmas_words_column.append('свадьба') 
    elif 'образование' in i:
        lemmas_words_column.append('образование') 
    elif 'автомобиль' in i:
        lemmas_words_column.append('автомобиль') 
    elif 'жилье' in i  or 'недвижимость' in i:
        lemmas_words_column.append('недвижимость') 
df['purpose'] = lemmas_words_column

#проверяем, появились ли новые дубликаты после лемматизации
print()
if df.duplicated().sum() == 0:
    print('Дубликатов нет')
else:
    print('Не все дубликаты удалены')

    
#удаляем новые дубликаты
df = df.drop_duplicates().reset_index(drop = True)    

Counter({'автомобиль \n': 964, 'свадьба \n': 785, 'на   проведение   свадьба \n': 760, 'сыграть   свадьба \n': 756, 'операция   с   недвижимость \n': 671, 'покупка   коммерческий   недвижимость \n': 655, 'покупка   жилье   для   сдача \n': 647, 'операция   с   коммерческий   недвижимость \n': 644, 'операция   с   жилье \n': 642, 'покупка   жилье \n': 636, 'покупка   жилье   для   семья \n': 636, 'жилье \n': 636, 'недвижимость \n': 628, 'строительство   собственный   недвижимость \n': 626, 'операция   со   свой   недвижимость \n': 626, 'строительство   жилой   недвижимость \n': 620, 'строительство   недвижимость \n': 619, 'покупка   свой   жилье \n': 619, 'покупка   недвижимость \n': 615, 'ремонт   жилье \n': 602, 'покупка   жилой   недвижимость \n': 601, 'на   покупка   свой   автомобиль \n': 502, 'заниматься   высокий   образование \n': 493, 'сделка   с   подержанный   автомобиль \n': 480, 'на   покупка   подержать   автомобиль \n': 472, 'свой   автомобиль \n': 472, 'на   покупка   ав

С помощью pymystem3 произвели лемматизацию столбца 'purpose' и с её помощью выделили 4 цели получения кредита.

### Категоризация данных.

Добавим дополнительные столбцы, разбив клиентов на группы по доходу, возрасту и наличию детей. Для лучшей категоризации данных и создания сводных таблиц.

In [16]:
#создадим функцию для категоризации количества детей
def children_group(children):
    if children < 1:
        return 'бездетный'
    return 'с детьми'
#добавим столбец в df
df['children_group'] = df['children'].apply(children_group)

#разделим уровни дохода на 4 группы: до 1/2 медианной, от 1/2 до медианной, от медианной да 1,5 медианных и от 1,5 медианных, присвоим этим значениям дохода переменные
median_income = df['total_income'].median()

#создадим функцию для определения уровня дохода
def income_group(income):
    if income < median_income/2:
        return 'низкий'
    if income < median_income:
        return 'средний'
    if income < median_income*1.5:
        return 'выше среднего'
    return 'высокий'

df['income_group'] = df['total_income'].apply(income_group)

#разделим клиентов на возрастные группы
def age_group(age):
        if age < 35:
                return 'молодой'
        if age <= 64:
                return 'средний возраст'
        return 'пенсионер'
    
df['age_group'] = df['dob_years'].apply(age_group)

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [17]:
df.pivot_table(index=['children_group'], columns= 'age_group', values='debt')

age_group,молодой,пенсионер,средний возраст
children_group,,,
бездетный,0.109463,0.055952,0.06855
с детьми,0.110371,0.050000,0.08139


Просроченные платежи по кредиту у клиентов возрастом до 35 лет возникают независимо от наличия детей.

У клиентов возраст которых от 35 до 64 лет, наличие детей увеличивает риск просрочек по кредиту практически на 1,5%. Это связано с тем, что клиенты данной возрастной группы, как правило являются достаточно сильно закредитованными, и дополнительные расходы на подрастающих детей увеличивают риск просрочки платежа.

Клиенты пенсионного возраста с детьми наоборот, менее подвержены риску дефолта, т.к. дети скорее всего трудоспособного возраста и могут оказать финансовую поддержку заёмщику в случае непредвиденных ситуаций.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [18]:
df_familystatus_pivot = (df.pivot_table(index=['family_status'], columns= 'gender', values='debt'))
df_familystatus_pivot['мean'] =  (df_familystatus_pivot['F'] + df_familystatus_pivot['M'])/2
df_familystatus_pivot

gender,F,M,мean
family_status,,,
Не женат / не замужем,0.069412,0.145695,0.107554
в разводе,0.066017,0.089494,0.077756
вдовец / вдова,0.058153,0.200000,0.129076
гражданский брак,0.082677,0.117465,0.100071
женат / замужем,0.069696,0.089058,0.079377


Процент просроченных кредитов выше у клиентов не состоящих в браке либо у вдов/вдовцов. При чём мужчины более подвержены риску дефолта кредита чем женщины.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [19]:
df.pivot_table(index=['income_group'], columns= 'income_type', values='debt')

income_type,госслужащий,компаньон,пенсионер,сотрудник
income_group,,,,
высокий,0.046667,0.062124,0.062356,0.083507
выше среднего,0.044444,0.078515,0.061224,0.095761
низкий,0.059406,0.078947,0.050420,0.081395
средний,0.077966,0.082305,0.058215,0.105605


Клиенты с доходом 'средний' и 'выше среднего' наиболее подвержены риску просрочки по кредиту. При этом наиболее часто просрочку допускают клиенты с типом занятости 'сотрудник'

#### Как разные цели кредита влияют на его возврат в срок?

In [20]:
df.pivot_table(index=['purpose'], columns= 'education', values='debt')

education,высшее,неоконченное высшее,среднее
purpose,,,
автомобиль,0.064390,0.104575,0.103076
недвижимость,0.049049,0.090186,0.081829
образование,0.059913,0.091603,0.104861
свадьба,0.045455,0.076923,0.091925


Наибольший процент просроченных кредитов приходится на кредиты взятые на образование и на покупку автомобиля. При чём клиенты не имеющие высшего образования имеют больший риск просрочки по кредиту

###  Общий вывод

Исходя из проведенного анализа, можно сделать следующие выводы:
Наличие детей влияет на риск невозврата кредита. Причём для клиентов среднего возраста (от 35 до 64 лет) наличие детей увеличивает процент просроченных платежей по кредиту с 6.9%п.п. до 8.1% . А у клиентов "пенсионного" возраста(от 64 лет) наоборот незначительно уменьшает на 0,6%.

Тип сеймейного положения клиента также имеет влияние на процент просроченных платежей по кредиту. Клиенты, состоящие в гражданском браке и клиенты потерявшие супруга/супругу, более склонны к допущению просроченных платежей.

Клиенты со средним и выше среднего доходами чаще остальных допускают просроченные платежи по кредитам. Скорее всего это связано с хорошей одобряемостью кредитов со стороны банков (в отличии от клиентов с низким доходом), что влечёт под собой сильную кредитную нагрузку, но в случае непредвиденных обстоятельств, они менее защищены в отличие от клиентов с высоким доходом. Причём клиенты работающие по найму наиболее подвержены просрочкам по кредиту.

Цели кредита на образование и на покупку автомобиля более рисковые в плане вероятности допущения просрочек нежели кредиты взятые на покупку недвижимости(как правило обеспеченные) и на свадьбу. Причём риск невозврата кредита во всех категориях является максимальным у клиентов со средним образованием и уменьшается у клиентов с высшим.

По результатам проведенного анализа, рекомендуется учесть вышестоящие выводы при построении скоринговой модели.

